# Описание проекта


Вам нужно защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.


- [1. Загрузка данных](#1)
- [2. Умножение матриц](#2)
- [3. Алгоритм преобразования](#3)
- [4. Проверка алгоритма](#4)
- [Общий вывод](#5)

## 1. Загрузка данных<a id="1"></a>

#### 1.1 Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#### 1.1 Изучение данных

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

Изучим данные - выведем первые три строки датафрейма, оценим количество объектов и признаки в нём, проверим данные на наличие пропусков/дупликатов:

In [3]:
display(df.head(3))
print(df.info())
df.duplicated().value_counts()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


False    4847
True      153
dtype: int64

**Вывод:**

- Датасет содержит 4 признака (пол, возраст и зарплата застрахованного, количество членов его семьи) и 1 целевой признак - количество страховых выплат клиенту за последние 5 лет. Данные признаки являются количественными.
- Тип данных признака "Возраст" целесообразно изменить на **int64**.
- Пропуски в данных отсутствуют
- Присутствует 153 задублированных объекта.
- Названия столбцов следует изменить для удобства работы.

#### 1.2 Обработка данных

Переименуем названия признаков:

In [4]:
df.columns = ['gender', 'age', 'salary', 'family_count', 'payout']
df.head(3)

,gender,age,salary,family_count,payout
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


Заменим тип данных признака **age** на **int64**:

In [5]:
df.age = df.age.astype('int')
df.age.dtype

dtype('int64')

Избавимся от задублированных объектов:

In [6]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().value_counts()

False    4847
dtype: int64

**Вывод:** данные подготовлены для дальнейшей работы.

## 2. Умножение матриц<a id="2"></a>

#### Ответим на вопрос и обоснуем решение.
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака
- $a$ — предсказания целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)


- $X'$ — матрица признаков после преобразования
- $w'$ — вектор весов линейной регрессии после преобразования
- $a'$ — предсказания целевого признака после преобразования

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Математическое обоснование:**

Формула матрицы признаков после преобразования:

$$
X' = XP
$$

Выведем формулу обучения (вектор весов) после преобразования:

$$
w' = ((XP^T) XP)^{-1} (XP)^T y
$$

Раскроем скобки данной формулы:

$$
w' = (P^TX^TXP)^{-1} P^TX^T y
$$

Можно заметить, что в формуле присутствуют переменные из изначальной формулы обучения. Для удобства, преобразуем выражение:


$$
w' = P^{-1} (X^{T} X)^{-1} (P^{T})^{-1} P^{T}X^{T}y
$$

Зная свойства квадратных матриц (Матрица $А^{-1}$ называется обратной матрицей по отношению к матрице А, если $А*А^{-1} = Е$, где $Е$ — единичная матрица n-го порядка) и то, что если любую матрицу $A$ умножить на единичную, получится эта же матрица $A$, следующим выражением можно пренебречь, т.к. в результате мы получаем единичную матрицу $E$, при умножении на которую умножаемая матрица не изменится:

$$
(P^T)^{-1}P^T = E
$$

В результате, формула принимает вид:

$$
w' = P^{-1} (X^{T} X)^{-1} X^{T}y
$$

Из данной формулы можно выделить изначальную формулу обучения линейной регрессии $w = (X^T X)^{-1} X^T y$ :

$$
w' = P^{-1}w
$$

Выведем формулу предсказания целевого признака после преобразования, используя формулы, выведенные ранее:

$$
a' = X'w' = XPP^{-1}w
$$

Можно заметить, что происходит умножение матрицы $P$ на обратную ей матрицу $P^{-1}$ и, опять же, зная свойства квадратных матриц, данное произведение никак не повлияет на результат и им можно пренебречь:

$$
a' = X'w' = Xw = a
$$

Как мы видим, в результате выведения формул можно прийти к выводу, что $a = a'$



**Ответ:** при умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

## 3. Алгоритм преобразования<a id="3"></a>

**Алгоритм**

1) создаём случайную квадратную матрицу $P$<br>
2) проверяем матрицу $P$ на обратимость<br>
3) умножаем матрицу признаков $X$ на матрицу $P$ и получаем преобразованную матрицу $X'$, содержащую закодированную информацию по ключу $P$<br>

**Обоснование**

Подобный способ позволяет в нужной мере преобразовать персональные данные, что не позволит интерпретировать их.

Как было выяснено в предыдущем шаге, подобный алгоритм, в идеальных условиях, не способен повлиять на качество работы модели.

## 4. Проверка алгоритма<a id="4"></a>

#### 4.1 Реализация алгоритма

Создадим случайную квадратную матрицу, используя функцию rand() из модуля random библиотеки numpy:

In [7]:
np.random.seed(42)
p = np.random.rand(4, 4)
print(p)

[[0.37454012 0.95071431 0.73199394 0.59865848]
 [0.15601864 0.15599452 0.05808361 0.86617615]
 [0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451]]


Проверим созданную матрицу на обратимость

In [8]:
np.linalg.inv(p)

array([[-0.32076901, -0.12766508,  0.06141427,  1.32518674],
       [ 0.35151041, -1.88500014,  1.65560045, -1.0003883 ],
       [ 1.14080312,  1.3467702 , -2.0407373 ,  0.70794071],
       [-0.08202687,  1.42666425, -0.17238177, -0.10600441]])

Матрица $P$ обратима.

Инициализируем две переменные, содержащие признаки и целевой признак:

In [9]:
features = df.drop('payout', axis=1)
target = df.payout

Инициализируем переменную, содержащую матрицу признаков:

In [10]:
features_matrix = features.values
print(features_matrix)

[[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00]]


Закодируем признаки, умножив матрицу признаков на случайную матрицу, созданную ранее:

In [11]:
encoded_features = features_matrix @ p
print(encoded_features)

[[29822.90832948 35127.95868743  1024.28616408 48143.82395221]
 [22850.37974634 26914.1460433    785.06445437 36896.60188937]
 [12627.93978718 14874.04797481   433.95880496 20393.22600363]
 ...
 [20382.58415619 24007.20495591   699.33967881 32897.63432023]
 [19662.76516213 23158.99290502   675.66827178 31736.25691292]
 [24410.84498147 28753.27755851   838.27062846 39403.37499285]]


Данные преобразованы, приступим к проверке и сравнению работы модели на преобразованных и исходных признаках.

#### 4.2 Проверка работы модели на исходных данных

Разделим данные на обучающую и проверочную выборки в соотношении 75:25:

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                              train_size=0.75, 
                                                                              test_size=0.25, 
                                                                              random_state=42)

#проверим размеры выборок:
print('Обучающая:',features_train.shape, target_train.shape)
print('Проверочная:',features_test.shape, target_test.shape)

Обучающая: (3635, 4) (3635,)
Проверочная: (1212, 4) (1212,)


Обучим модель линейной регрессии без подбора гиперпараметров и выведем метрику R2:

In [13]:
model = LinearRegression()

model.fit(features_train, target_train)

r2_initial = r2_score(target_test, model.predict(features_test))
print('Метрика R2 на исходных данных', r2_initial)

Метрика R2 на исходных данных 0.44346330831611536


#### 4.3 Проверка работы модели на преобразованных данных

In [14]:
features_train, features_test, target_train, target_test = train_test_split(encoded_features, target, 
                                                                              train_size=0.75, 
                                                                              test_size=0.25, 
                                                                              random_state=42)

#проверим размеры выборок:
print('Обучающая:',features_train.shape, target_train.shape)
print('Проверочная:',features_test.shape, target_test.shape)

Обучающая: (3635, 4) (3635,)
Проверочная: (1212, 4) (1212,)


In [15]:
model = LinearRegression()

model.fit(features_train, target_train)

r2_encoded = r2_score(target_test, model.predict(features_test))
print('Метрика R2 на исходных данных', r2_encoded)

Метрика R2 на исходных данных 0.4434633083154135


In [16]:
print('Разница между метриками R2 на исходных и преобразованных данных:', abs(r2_encoded - r2_initial))

Разница между метриками R2 на исходных и преобразованных данных: 7.01882996168024e-13


# Общий вывод<a id="5"></a>

- После загрузки и изучения данных мы математически доказали, что качество работы линейной регрессии не изменится при умножении признаков на обратимую матрицу
- Используя предложенный алгоритм, были построены две модели линейной регрессии, позволившие уже практически подтвердить теоретические расчёты.
- Разница между метрикой R2 на исходных данных и преобразованных близка к нулю и мы можем сделать вывод, что алгоритмы линейной регрессии нечувствительны к умножению признаков на обратимую матрицу.